In [1]:
import json
#import pydevd
#pydevd.settrace('localhost', port=49309, stdoutToServer=True, stderrToServer=True)
import tensorflow as tf

import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict, defaultdict
from six.moves import xrange
#from __pynauty__ import graph
#import pynauty.graph
import pynauty
import time
import tensorflow as tf
import networkx as nx
import numpy as np
import pynauty as nauty
from multiprocessing import Pool
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import os
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.metrics import accuracy_score

from PatchyTools import Dataset
from PatchyTools import utils
from PatchyTools import Patchy_san

### Functions

In [2]:
mutag = Dataset.Dropbox('MUTAG')
df_edge_label = mutag.get_edge_label()
df_graph_ind = mutag.get_graph_ind()
df_adj = mutag.get_adj()
df_node_label = mutag.get_node_label()
df_node_label = pd.concat([df_node_label, df_graph_ind.graph_ind], axis=1)
del df_graph_ind


In [3]:
num_features = len(df_node_label.label.unique())
data_in_patchysan = Patchy_san.main(WIDTH_W=18, RECEPTIVE_FIELD_SIZE_K=10, datasetname='MUTAG')
data_in_patchysan.shape[3]
final_labels = mutag.get_graph_label().graph_label.values
final_labels = pd.get_dummies(final_labels).values

### CNN  

In [4]:
CONV_FIELD_SIZE=10
early_stopping = 60
STOPPING_LOOKBACK_NUM =20
test_range =10
LEARNING_RATE = 0.0005
EPSILON=1e-08


In [5]:

##########################################
# Build a Multilayer Convolutional Network
###########################################

from __future__ import absolute_import, unicode_literals
import tensorflow as tf

FeatureNum = data_in_patchysan.shape[3]
PixcelSize = data_in_patchysan.shape[1]* data_in_patchysan.shape[2]

W = tf.Variable(tf.zeros([PixcelSize, FeatureNum]))
b = tf.Variable(tf.zeros([FeatureNum]))

"""
Weight Initialization

To create this model, we're going to need to create a lot of weights and biases.
One should generally initialize weights with a small amount of noise for symmetry breaking,
and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize
them with a slightly positive initial bias to avoid "dead neurons." Instead of doing this repeatedly
while we build the model, let's create two handy functions to do it for us.
"""


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

"""
Convolution and Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling operations.
How do we handle the boundaries? What is our stride size? In this example,
we're always going to choose the vanilla version. Our convolutions uses a stride of one
and are zero padded so that the output is the same size as the input. Our pooling is plain old
max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.
"""


def conv2d(x, W):
    #sprint('x.shape, W.shape', x.shape, W.shape)
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 1, 1, 1],
                          strides=[1, 1, 1, 1], padding='SAME')


x_image = tf.placeholder("float", shape=[None, 18, 10, 7]) #tf.reshape(data_in_patchysan, [-1, 18, 5, 7])
y_ = tf.placeholder("float", shape=[None,2])



"""
1st layer
patch size : 10x10 
input channel : F
output channel : 16
"""

W_conv1 = weight_variable([CONV_FIELD_SIZE,CONV_FIELD_SIZE,
                           FeatureNum, 
                           16])

b_conv1 = bias_variable([16])

temp = conv2d(x_image, W_conv1) + b_conv1
h_conv1 = tf.nn.relu(temp)

"""
1st layer
patch size : 5x5
input channel : 8
output channel : 8
"""

W_conv2 = weight_variable([CONV_FIELD_SIZE, CONV_FIELD_SIZE, 16, 8])
b_conv2 = bias_variable([8])
h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
#h_pool2 = max_pool_2x2(h_conv2)

"""

Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow
processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors,
multiply by a weight matrix, add a bias, and apply a ReLU.
"""

#num = np.product([i for i in h_conv2.shape[1:]])
num = 8 * data_in_patchysan.shape[1] *  data_in_patchysan.shape[2]
h_conv2_flat = tf.reshape(h_conv2, [-1, num])

W_fc1 = weight_variable([num, 128])
b_fc1 = bias_variable([128])
h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)

"""
Dropout
To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder
for the probability that a neuron's output is kept during dropout. This allows us to turn dropout
on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically
handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.
"""
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

"""
Readout Layer
Finally, we add a softmax layer, just like for the one layer softmax regression above.
"""
num_final_class = final_labels.shape[1]
W_fc2 = weight_variable([128, num_final_class])
b_fc2 = bias_variable([num_final_class])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [6]:
h_conv2.shape[1:]

TensorShape([Dimension(18), Dimension(10), Dimension(8)])

In [7]:
"""
Training and evaluation 
TensorFlow
"""

cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE, 
                                    epsilon=EPSILON, use_locking=False,
                                    name='Adam'
                                   ).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  #accuracy = tf.metrics.accuracy(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

sess = tf.InteractiveSession()


In [8]:

cost_val = []
test_accuracy_list=[]


'''
Preapre k-folds index
'''
rs = ShuffleSplit(n_splits=test_range, random_state=0)
rs.get_n_splits(data_in_patchysan)
k_folds ={}
kf=0
for train_index, test_index in rs.split(data_in_patchysan):
    k_folds[kf] =[train_index, test_index]
    #print("TRAIN:", train_index, "TEST:", test_index)
    kf+=1

'''
Conduct experiments
'''

for t in range(test_range):    
    #seed = t
    sess.run(tf.initialize_all_variables())
    tf.global_variables_initializer()
    
    test_index = k_folds[t][1]    
    train_index = k_folds[t][0]
    valid_index = k_folds[t][0][:19]
    train_index = k_folds[t][0][19:]
    
    X_train = data_in_patchysan[train_index]
    y_train = final_labels[train_index]    
    X_valid = data_in_patchysan[valid_index]
    y_valid = final_labels[valid_index]
    X_test = data_in_patchysan[test_index]
    y_test = final_labels[test_index]
    
    for i in range(120):    
        if i > early_stopping and cost_val[-1] > np.mean(cost_val[-STOPPING_LOOKBACK_NUM-1:-1]):
            test_accuracy = accuracy.eval(feed_dict={x_image: X_test, y_: y_test, keep_prob: 1.0})
            print ('Early Stopping with step at {}'.format(i, cost, test_accuracy) )
            break
        traincost = accuracy.eval(feed_dict={ x_image: X_train, y_: y_train,keep_prob: 1.0})
        cost = accuracy.eval(feed_dict={x_image: X_valid, y_: y_valid,keep_prob: 1.0})
        cost_val.append(cost)
        if i %20==0: 
            print ("step %d, train: %g , valid: %s " % (i, traincost, cost))    
        
        train_step.run(feed_dict={x_image: X_train, y_: y_train, keep_prob: 0.5})
    
    test_accuracy = accuracy.eval(feed_dict={x_image: X_test, y_: y_test, keep_prob: 1.0})
    test_accuracy_list.append(test_accuracy)
    print( "test accuracy %g" % accuracy.eval(feed_dict={x_image: X_test, y_: y_test, keep_prob: 1.0}))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, train: 0.64 , valid: 0.736842 
step 20, train: 0.973333 , valid: 0.789474 
step 40, train: 1 , valid: 0.789474 
step 60, train: 1 , valid: 0.789474 
Early Stopping with step at 61
test accuracy 0.894737
step 0, train: 0.666667 , valid: 0.526316 
step 20, train: 0.953333 , valid: 0.631579 
step 40, train: 1 , valid: 0.631579 
step 60, train: 1 , valid: 0.631579 
Early Stopping with step at 61
test accuracy 0.789474
step 0, train: 0.713333 , valid: 0.684211 
step 20, train: 0.946667 , valid: 0.789474 
step 40, train: 1 , valid: 0.736842 
step 60, train: 1 , valid: 0.789474 
Early Stopping with step at 61
test accuracy 0.894737
step 0, train: 0.36 , valid: 0.210526 
step 20, train: 0.926667 , valid: 0.842105 
step 40, train: 1 , valid: 0.842105 
step 60, train: 1 , valid: 0.894737 
Early Stopping with step at 61
test accuracy 0.894737
step 0, train: 0.64 , valid: 0.736842 
step 20, train: 0.953333 , valid: 0

In [9]:
np.mean(test_accuracy_list), '+-', np.std(test_accuracy_list)

(0.85263157, '+-', 0.045883127)

In [10]:
#tf.train.AdamOptimizer(learning_rate=rate_use, beta1=beta_1, beta2=beta_2, epsilon=eta, 
#                       use_locking=False, name='Adam').minimize(objective_value, global_step=step)
